In [7]:
# Load API key

# Note the file with the API key is not in the GitHub repo.
# Keys need to be read from a local directory
with open("jwCensusApi", "r") as i:
    key = i.read()

In [10]:
import json
import requests
import pandas as pd
import matplotlib as plt

In [24]:
'''
Current Population Survey Annual Social and Economic Supplement
https://api.census.gov/data.html

Vars: https://api.census.gov/data/2021/cps/asec/mar/variables.html
'''

endpoint = "http://api.census.gov/data/2021/cps/asec/mar" # state geographies
ex1_url = "https://api.census.gov/data/2021/cps/asec/mar?get=NOW_COV,MARSUPWT,A_MARITL&for=state:35&A_HGA=39"
ex2_url = "https://api.census.gov/data/2021/cps/asec/mar?tabulate=weight(MARSUPWT)&col+NOW_COV&row+A_MARITL&for=state:35&A_HGA=39"

In [25]:
api_request_url = ex1_url + "&key=" + key
api_request = requests.get(api_request_url)

In [26]:
api_json = api_request.json()

In [34]:
# print(json.dumps(api_json, indent=4)) # long output

### Q: How to get info on variables via API?

We can look it up online, but we may also get this info dynamically.

In [35]:
var = "NOW_COV"
var_def_url = endpoint + "/variables/" + var

In [36]:
var_def_request = requests.get(var_def_url)

In [37]:
var_json = var_def_request.json()

In [38]:
print(json.dumps(var_json, indent=4))

{
    "name": "NOW_COV",
    "label": "Currently covered by health insurance coverage",
    "predicateType": "int",
    "group": "N/A",
    "limit": 0,
    "suggested-weight": "MARSUPWT",
    "values": {
        "item": {
            "2": "No",
            "1": "Yes"
        }
    }
}


In [ ]:
'''
Variables of interest

A_HGA - educational attainment
AGE1 - age
AGI - federal adjusted gross income

NM FIPS ID = 35
'''